In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('SUWON_S_DATA_TABLE_GENDER_SUM.csv')

In [3]:
df

,TA_YMD,HOUR,DAY,DONG,AMT,CNT,UNIT,TEMP,RAIN
0,20230101,1,7,고등동,65063,7,9295,2.10,0.0
1,20230101,2,7,고등동,14878,4,3720,-0.20,0.0
2,20230101,3,7,고등동,77671,20,3884,-2.25,0.0
3,20230101,4,7,고등동,452897,52,8710,-3.20,0.0
4,20230101,5,7,고등동,972179,92,10567,-4.10,0.0
...,...,...,...,...,...,...,...,...,...
101508,20251031,6,5,화서2동,886440,155,5719,11.65,1.5
101509,20251031,7,5,화서2동,1319747,105,12569,11.65,0.0
101510,20251031,8,5,화서2동,3662364,125,29299,10.70,0.0
101511,20251031,9,5,화서2동,1999679,51,39209,9.35,0.0


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [16]:
en_ymd = encoder.fit_transform(df['TA_YMD'])
en_ymd

array([   0,    0,    0, ..., 1034, 1034, 1034], dtype=int64)

In [15]:
en_dong = encoder.fit_transform(df['DONG'])
en_dong

array([0, 0, 0, ..., 9, 9, 9])

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 데이터 불러오기
df = pd.read_csv('SUWON_S_DATA_TABLE_GENDER_SUM.csv')

# 날짜 처리
df['date'] = pd.to_datetime(df['TA_YMD'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday

# DONG 컬럼 코드화
dong_encoder = LabelEncoder()
df['DONG_CODE'] = dong_encoder.fit_transform(df['DONG'])

# 기존 문자 컬럼 제거
df.drop(columns=['DONG'], inplace=True)

# feature / target 분리
X = df.drop(columns=['UNIT', 'date'])
y = df['UNIT']

# 학습/검증 분리 (날짜 순서 유지 권장)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, test_size=0.2
)

# 모델 학습
model = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    random_state=42
)
model.fit(X_train, y_train)

# 예측
pred = model.predict(X_test)

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("MAE :", mean_absolute_error(y_test, pred))
print("RMSE:", mean_squared_error(y_test, pred, squared=False))
print("R2  :", r2_score(y_test, pred))

MAE : 975.900141040262
RMSE: 1946.6314260180718
R2  : 0.9441827379667121


In [52]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import os
from dotenv import load_dotenv
load_dotenv('./../01_python/.env')
key = os.getenv('RAIN_ID')

def get_rainfall_T(bdate):
    return_str = ''
    btm='0900'
    nx='61'
    ny='121'
    url1 = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?ServiceKey={key}&base_date={bdate}&base_time={btm}&nx={nx}&ny={ny}'
    response = requests.get(url1)
    soup = BeautifulSoup(response.text, 'xml')
    soup
    if soup.resultCode.text == '00':
        for item in soup.items:
            if item.category.text == 'T1H':
                return_str = item.obsrValue.text
            #if item.category.text == 'RN1':
            #    print('강수:',item.obsrValue.text)
    else:
        print('오류:', soup.resultMsg.text )
        return_str = soup.resultMsg.text
    return return_str

def get_rainfall_R(bdate):
    return_str = ''
    btm='0900'
    nx='61'
    ny='121'
    url1 = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?ServiceKey={key}&base_date={bdate}&base_time={btm}&nx={nx}&ny={ny}'
    response = requests.get(url1)
    soup = BeautifulSoup(response.text, 'xml')
    soup
    if soup.resultCode.text == '00':
        for item in soup.items:
            if item.category.text == 'RN1': # 강수형태(PTY) 코드 : 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)
                return_str = item.obsrValue.text
    else:
        print('오류:', soup.resultMsg.text )
        return_str = soup.resultMsg.text
    return return_str

today = datetime.now()
today_str = today.strftime('%Y%m%d')

temp_today = get_rainfall_T(today_str)
rain_today = get_rainfall_R(today_str)
print(temp_today, rain_today)

3.2 0
